#Titanic - Machine Learning from a Disaster


In [19]:
import pandas
import numpy
from sklearn import neighbors
from sklearn.neighbors import DistanceMetric 
from pprint import pprint
from math import sqrt
import operator

MY_TITANIC_TRAIN ='./sample_data/train.csv'
MY_TITANIC_TEST = './sample_data/test.csv'
titanic_dataframe = pandas.read_csv(MY_TITANIC_TRAIN)
print('length: {0} '.format(len(titanic_dataframe)))
titanic_dataframe.head(2)






length: 891 


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [20]:
ave_age = 29.699118
titanic_dataframe['Age'].fillna(ave_age, inplace=True)





We replaced the NA age values with the mean ages to complete data set.

In [21]:
titanic_dataframe = titanic_dataframe.dropna()

titanic_dataframe['Sex'].unique()
titanic_dataframe['Gender'] = titanic_dataframe['Sex'].map({'female': 0, 'male': 1}).astype(int)
titanic_dataframe['Embarked'].unique()
titanic_dataframe['Port'] = titanic_dataframe['Embarked'].map({'C':1, 'S':2, 'Q':3}).astype(int)

titanic_dataframe.drop(['Sex', 'Embarked', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

In [22]:
cols = titanic_dataframe.columns.tolist()
cols = [cols[1]] + cols[0:1] + cols[2:]
titanic_dataframe = titanic_dataframe[cols]

train_data = titanic_dataframe[cols[2: ]]
train_target = titanic_dataframe[cols[1]]

pprint('column_list: {0}'.format(cols))
print(train_data.head(2))

("column_list: ['Survived', 'PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', "
 "'Fare', 'Gender', 'Port']")
   Pclass   Age  SibSp  Parch     Fare  Gender  Port
1       1  38.0      1      0  71.2833       0     1
3       1  35.0      1      0  53.1000       0     2


In [23]:
titanic_dataframe.head(5)

,Survived,PassengerId,Pclass,Age,SibSp,Parch,Fare,Gender,Port
1,1,2,1,38.0,1,0,71.2833,0,1
3,1,4,1,35.0,1,0,53.1000,0,2
6,0,7,1,54.0,0,0,51.8625,1,2
10,1,11,3,4.0,1,1,16.7000,0,2
11,1,12,1,58.0,0,0,26.5500,0,2


In [24]:
def hamming_distance(input_row, training_row):
    distance = list(
            # 1 if isinstance(input_row[i], str) & training_row[i] != input_row[i] else 0
            1 if input_row[key] != training_row[key] else 0 for key in factors
        # for i in range(len(input_row))
    )
    return sum(distance)

factors = [ 'Pclass', 'Port', 'Gender']
hamming = DistanceMetric.get_metric('hamming')

In [25]:
# for train_row in train:
#     hamming_distance(train_row)

In [26]:
def euclidian_distance(row, train_row):
    distance = list((row[key] - train_row[key])**2 for key in numericals)
                     # for i in range(len(input_row)) if isnumeric(row[i]) & isnumeric(train_row[i])
               # )
    return sqrt(sum(distance))

numericals = ['Age', 'SibSp', 'Parch', 'Fare']
euclidian = DistanceMetric.get_metric('euclidean')

In [27]:
def knn(input_row, train, k):
    distance = []
    # for train_row in train:
    for row in train.iterrows():
        if row[0] != input_row[0]:
        # that means row = tuple and i can get what i want by saying row[1]['Column Name']            
            hamming = hamming_distance(input_row[1], row[1])
            euclidian = euclidian_distance(input_row[1], row[1])
            distance.append((hamming + euclidian, row[1]['Survived']))
#             import pdb; pdb.set_trace();
    distance.sort(key=operator.itemgetter(0))
    out = distance[ :k]
    return (input_row[0], 1 if sum(out[1]) > k//2 else 0)

who_dies = []
    
for row in titanic_dataframe.iterrows():
   who_dies.append(knn(row, titanic_dataframe, 10))
# knn(titanic_dataframe.iterrows(), titanic_dataframe, 10)

(1, 1)
(3, 0)
(6, 1)
(10, 1)
(11, 1)
(21, 0)
(23, 0)
(27, 1)
(31, 1)
(52, 0)
(54, 1)
(55, 0)
(62, 1)
(66, 0)
(75, 1)
(88, 0)
(92, 1)
(96, 1)
(97, 1)
(102, 0)
(110, 0)
(118, 1)
(123, 0)
(124, 0)
(128, 1)
(136, 1)
(137, 0)
(139, 0)
(148, 0)
(151, 1)
(166, 0)
(170, 1)
(174, 0)
(177, 1)
(183, 1)
(185, 0)
(193, 1)
(194, 0)
(195, 1)
(205, 1)
(209, 1)
(215, 1)
(218, 1)
(224, 1)
(230, 1)
(245, 1)
(248, 0)
(251, 0)
(252, 1)
(257, 0)
(262, 0)
(263, 1)
(268, 1)
(269, 1)
(273, 1)
(275, 1)
(284, 0)
(291, 1)
(292, 0)
(297, 1)
(298, 0)
(299, 1)
(303, 0)
(305, 1)
(307, 0)
(309, 0)
(310, 1)
(311, 0)
(318, 1)
(319, 1)
(325, 1)
(327, 0)
(329, 1)
(331, 0)
(332, 1)
(336, 1)
(337, 1)
(339, 0)
(340, 1)
(341, 1)
(345, 0)
(351, 0)
(356, 1)
(366, 1)
(369, 1)
(370, 1)
(377, 1)
(390, 1)
(393, 1)
(394, 1)
(412, 1)
(429, 0)
(430, 0)
(434, 1)
(435, 1)
(438, 1)
(445, 1)
(449, 1)
(452, 0)
(453, 1)
(456, 1)
(457, 0)
(460, 0)
(462, 0)
(473, 1)
(475, 0)
(484, 1)
(486, 0)
(487, 0)
(492, 0)
(496, 0)
(498, 1)
(504, 1)
(505,